In [2]:
import streamlit as st
import pandas as pd
import altair as alt
import numpy as np

In [7]:
f1 = pd.read_csv('data/chefmozaccepts.csv',encoding='ISO-8859-1')

In [8]:
files = ['chefmozaccepts',
         'chefmozcuisine',
         'chefmozhours4',
         'chefmozparking',
         'usercuisine',
         'userpayment',
         'userprofile',
         'rating_final',
         'geoplaces2']
dfs = dict()

for file in files:
    fl= 'data/'+file+'.csv'
    dfs[file] = pd.read_csv(fl,encoding='ISO-8859-1')

In [9]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [10]:
dfs['chefmozhours4']['hours'] = dfs['chefmozhours4']['hours'].str.strip()
dfs['chefmozhours4']['days'] = dfs['chefmozhours4']['days'].str.strip()
